<a href="https://colab.research.google.com/github/dannyweiland/Pinkbike-Market-Analysis/blob/main/UsedBikeMarket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PinkBike Buy/Sell Market Report: Enduro**

The script below pulls the data from every Enduro bike listing in the North America region of Pinkbike.com.

In [ ]:
#Import Packages
from pprint import pprint
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd



def main():
    category = 2
    bikes = []
    for i in range(1, 485):  #483 pages, checked PinkBike before runing this script.  I could make this variable more dynamic, but would rather spend the 2 seconds just checking Pinkbike and hardcoding this value.
        bikes.extend(scrape_page(category, i))

#Define output dataframe
    df = pd.DataFrame(bikes)
    print(df)
    df.to_csv('bikes_data.csv', index=False)



def scrape_page(category_id, page_number):
    response = requests.get(
        "https://www.pinkbike.com/buysell/list/",
        params={"region": 3, "category": category_id, "page": page_number},
        timeout=10,
    )

    response.raise_for_status()

    soup = bs(response.text, "html.parser")


    bikes = []
    for bike in soup.select(".bsitem"):
        info = {}
        info["title"] = bike.select_one(".bsitem-title a").text           #Item title
        info["price"] = bike.select_one(".bsitem-price").text.strip()     #Item price
        info["location"] = bike.select_one(".flag").parent.text.strip()   #Item location
        link = bike.select_one(".bsitem-title a")["href"]
        info["link"] =  link


        #Now is where we get a lil tricky.  We need to extract the additional details that don't have a cool little custom tag like the others.  This code pulls all itemdetails.
        details_element = bike.select(".itemdetail")
        for detail in details_element:
          key = detail.select_one("b").text.strip().rstrip(':')  # Extract the key
          value = detail.contents[2].strip()  # Extract the value
          info[key] = value

        bikes.append(info)
    return(bikes)





if __name__ == "__main__":
    main()




                                                  title      price  \
0     2020 Pivot Firebird pro XT/XTR medium (inc AXS...  $4500 CAD   
1     2023 Yeti SB165 L T2 new frame with custom build!  $6450 USD   
2               2022 Specialized Enduro custom build S5  $5499 USD   
3                       2022 Devinci Spartan High Pivot  $4400 CAD   
4                         2023 Cannondale Jekyll Custom  $8999 CAD   
...                                                 ...        ...   
9650                    2020 Scott Genius 900 Tuned AXS  $5500 USD   
9651                 2021 Specialized Turbo Levo Comp M  $7500 CAD   
9652           2019 Men's Stumpjumper EVO Comp Alloy 29  $2800 USD   
9653           2018 Specialized Enduro Elite 29/6Fattie  $3100 USD   
9654        2019 Kona Process 153 CR with some upgrades  $3500 USD   

                                  location  \
0         Duncan, British Columbia, Canada   
1         Durango, Colorado, United States   
2        schodack, Ne